In [1]:
import pandas as pd
from utils import *

In [2]:
df = pd.read_csv("output3.csv",index_col=0)
df = df[~df["mean"].duplicated()]
df["recent"] = df.index.str.slice(-1)
df = df[df["count"] > 20]
df.head(3)

,win_ratio,mean,count,recent
foot2-yz0-~top1-1,0.750000,1.020656,32,1
minute-yz0-~yz1-top1-opentop1-4,0.743590,1.022049,39,4
small_volume-yz0-~yz1-top1-opentop1-4,0.736842,1.021491,38,4


In [7]:
valid_keys = ['~opentop2-jump1-foot0-~foot1-~foot2-~isnew-small_capq-minute-7', '~opentop2-~speedup2-foot0-~foot1-~foot2-~isnew-small_capq-minute-7']
sub_df = df[df.index.isin(valid_keys) == True]
valid_keys = remove_invalid(sub_df,7)
valid_keys

set()

In [ ]:
data

In [11]:
for i in range(2,8):
    sub_df = df[df["recent"] == str(i)]
    valid_keys = remove_invalid(sub_df,i)
    print valid_keys
    print len(valid_keys),len(sub_df),i
    print "--------"

/Users/sgcy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/sgcy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


set(['opentop2-~jump1-~speedup2-foot0-~foot1-~opentop1-top1-2', 'opentop2-speedup1-foot0-~opentop1-top1-2', '~opentop2-speedup1-~jump1-speedup2-jump2-~foot0-foot1-~foot2-small_volume-2'])
3 24 2
--------
set(['~opentop2-speedup1-jump2-foot0-~foot2-small_capq-small_volume-~opentop1-top1-3', 'speedup1-~jump2-~foot1-~foot2-small_volume-~yz1-top1-opentop1-3', '~opentop2-~speedup1-jump2-~foot0-~foot1-foot2-3', '~opentop2-~jump2-~foot0-small_capq-minute-small_volume-~yz1-top1-opentop1-3', 'opentop2-~jump1-speedup2-~foot1-~opentop1-top1-3', '~speedup1-jump1-~foot0-~foot1-minute-small_volume-3'])
6 218 3
--------
set(['~speedup1-speedup2-~foot1-~foot2-small_capq-small_volume-~opentop0-top0-~opentop1-top1-4', '~opentop2-~speedup1-jump2-~foot0-~foot1-small_capq-small_volume-4', '~opentop2-~speedup1-~foot2-minute-small_volume-~yz1-top1-opentop1-4', '~opentop2-~speedup1-~foot0-~foot1-foot2-small_capq-4', '~speedup1-~jump2-minute-small_volume-~yz1-top1-opentop1-4'])
5 390 4
--------
set(['minute-sm

In [10]:
def remove_invalid(output_df,day):
    keys = output_df.index
    valid_keys = set(keys)
    for keyA in keys:
        if not keyA in valid_keys:
            continue
        filterA = getFilter(keyA)
        filter_recent = recent(day,data_df)
        for keyB in keys:
            if not keyB in valid_keys or keyA == keyB:
                continue
#             print keyA + "--" + keyB
            filterB = getFilter(keyB)
            sub_dfA = data_df[filter_recent & filterA & ~filterB & ~yz2]
            sub_dfB = data_df[filter_recent & filterB & ~filterA & ~yz2]
            to_remove,to_break = shouldRemove(keyA,keyB,sub_dfA,sub_dfB,output_df)
            if to_remove is not None:
                valid_keys.remove(to_remove)
            if to_break:
                break            
    return valid_keys

In [5]:
def shouldRemove(keyA,keyB,sub_dfA,sub_dfB,output_df):
    if is_valid(sub_dfA) and not is_valid(sub_dfB):
        return keyB,False
    elif not is_valid(sub_dfA) and is_valid(sub_dfB):
        return keyA,True
    elif not is_valid(sub_dfA) and not is_valid(sub_dfB):
        if output_df["count"][keyA] > output_df["count"][keyB]:
            return keyB,False
        elif output_df["count"][keyA] < output_df["count"][keyB]:
            return keyA,True
        else:
            if keyA.split("-") < keyB.split("-"):
                return keyB,False
            elif keyA.split("-") > keyB.split("-"):
                return keyA,True
            else:
                if output_df["win_ratio"][keyA] >= output_df["win_ratio"][keyB]:
                    return keyB,False
                else:
                    return keyA,True
    else:
        return None,False

In [6]:
def is_valid(sub_df):
    sub_df = sub_df.sort_values("minute")
    sub_df = sub_df.groupby("date").first()
    count = len(sub_df)
    mean = sub_df.change.mean()
    if mean < 1.015 or count < 2:
        return False 
    win_df = sub_df[sub_df["change"] > 1.0]
    win_ratio = float(len(win_df))/len(sub_df)
    if count < 10:
        return mean > 1.025 and win_ratio > 0.75
    else:
        return mean > 1.02 and win_ratio > 0.6